In [7]:
import cv2 as cv
import numpy as np

In [8]:
lena = cv.imread('lena.png')
lena = cv.cvtColor(lena, cv.COLOR_BGR2GRAY)

In [9]:
def Gaussian(_noisedImage, kernelSize):
	return cv.blur(_noisedImage, (kernelSize, kernelSize))


def Median(_noisedImage, kernelSize):
	if kernelSize % 2 == 0:
		kernelSize += 1
	return cv.medianBlur(_noisedImage, kernelSize)


def Filter2D(_noisedImage, kernelSize):
	return cv.filter2D(_noisedImage, cv.CV_8U,
					   np.ones((kernelSize, kernelSize), dtype=np.float64) / (kernelSize * kernelSize))


def Sobel(_noisedImage, kernelSize):
	if kernelSize % 2 != 1:
		kernelSize += 1
	return cv.Sobel(cv.blur(_noisedImage, (kernelSize, kernelSize)), cv.CV_64F, 1, 1, ksize=kernelSize)


def Laplacian(_noisedImage, kernelSize):
	if kernelSize % 2 != 1:
		kernelSize += 1
	return cv.Laplacian(cv.blur(_noisedImage, (kernelSize, kernelSize)), cv.CV_64F, ksize=kernelSize)

In [10]:
def noise(img, noise_intensity, _is_normal):
	return cv.randn(np.zeros(img.shape), 0, noise_intensity) if _is_normal else cv.randu(np.zeros(img.shape),
																						 -noise_intensity,
																						 noise_intensity)


def addNoiseToImage(img, noise_intensity, _is_normal):
	return np.uint8(np.clip(cv.add(np.float64(img), noise(img, noise_intensity, _is_normal)), 0, 255))


def filtrateImage(_noisedImage, _typeOfFilter, kernelSize):
	if _typeOfFilter == "Gaussian":
		return Gaussian(_noisedImage, kernelSize)
	elif _typeOfFilter == "Median":
		return Median(_noisedImage, kernelSize)
	elif _typeOfFilter == "Filter2D":
		return Filter2D(_noisedImage, kernelSize)
	elif _typeOfFilter == "Sobel":
		return Sobel(_noisedImage, kernelSize)
	elif _typeOfFilter == "Laplacian":
		return Laplacian(_noisedImage, kernelSize)
	else:
		return None


is_normal = True
noiseIntensity = 40
sizeOfKernel = 3
typesOfFilters = list(("Gaussian", "Median", "Filter2D", "Sobel", "Laplacian"))
typeOfFilter = 0

noisedImage = addNoiseToImage(lena, 40, True)
filteredImage = filtrateImage(noisedImage, typesOfFilters[typeOfFilter], sizeOfKernel)

In [11]:
def update():
	global filteredImage
	filteredImage = filtrateImage(noisedImage, typesOfFilters[typeOfFilter], sizeOfKernel)
	cv.imshow("Result", filteredImage)
	cv.imshow("Noised", noisedImage)


def changeNoizeIntensity(val):
	global noisedImage
	global noiseIntensity
	noiseIntensity = val
	noisedImage = addNoiseToImage(lena, noiseIntensity, is_normal)
	update()


def changeTypeOfNoiseDistribution(val):
	global noisedImage
	global is_normal
	if val == 1:
		is_normal = True
	else:
		is_normal = False
	noisedImage = addNoiseToImage(lena, noiseIntensity, is_normal)
	update()


def changeKernelSize(val):
	global sizeOfKernel
	global filteredImage
	if val > 0:
		sizeOfKernel = val
		update()


def switchFilterType(val):
	global filteredImage
	global typeOfFilter
	typeOfFilter = val
	update()

In [12]:
cv.imshow("Result", lena)
cv.createTrackbar("Type of Filter", "Result", 0, 4, switchFilterType)
cv.createTrackbar("Kernel size", "Result", 3, 30, changeKernelSize)
cv.imshow("Noised", noisedImage)
cv.createTrackbar("Intensity", "Noised", 40, 500, changeNoizeIntensity)
cv.createTrackbar("Normal/Uniform", "Noised", 1, 1, changeTypeOfNoiseDistribution)
cv.waitKey(0)
cv.destroyAllWindows()